# Step-Back Prompting (Question-Answering)

One prompting technique called "Step-Back" prompting can improve performance on complex questions by first asking a "step back" question. This can be combined with regular question-answering applications by then doing retrieval on both the original and step-back question.

Read the paper [here](https://arxiv.org/abs/2310.06117)

See an excellent blog post on this by Cobus Greyling [here](https://cobusgreyling.medium.com/a-new-prompt-engineering-technique-has-been-introduced-called-step-back-prompting-b00e8954cacb)

In this cookbook we will replicate this technique. We modify the prompts used slightly to work better with chat models.

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

In [38]:
# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?"
    },
    {
        "input": "Jan Sindel’s was born in what country?", 
        "output": "what is Jan Sindel’s personal history?"
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [39]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"""),
    # Few shot examples
    few_shot_prompt,
    # New question
    ("user", "{question}"),
])

In [40]:
question_gen = prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [41]:
question = "was chatgpt around while trump was president?"

In [42]:
question_gen.invoke({"question": question})

'when was ChatGPT developed?'

In [43]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper


search = DuckDuckGoSearchAPIWrapper(max_results=4)

def retriever(query):
    return search.run(query)

In [44]:
retriever(question)

'True About this rating On Jan. 30, 2023, Twitter user @echo_chamberz tweeted that the OpenAI tool ChatGPT (Chat Generative Pre-Trained Transformer) declined a request to write an AI-generated... While impressive in many respects, ChatGPT also has some major flaws. ... [President\'s Name]," refused to write a poem about ex-President Trump, but wrote one about President Biden ... On Wednesday, a Twitter user posted screenshots of him asking OpenAI\'s chatbot, ChatGPT, to write a positive poem about former President Donald Trump, to which the chatbot declined, citing... During the Trump administration, Altman gained new attention as a vocal critic of the president. It was against that backdrop that he was rumored to be considering a run for California governor.'

In [45]:
retriever(question_gen.invoke({"question": question}))

"June 4, 2023 ⋅ 4 min read 124 SHARES 14K READS At the end of 2022, OpenAI introduced the world to ChatGPT. Since its launch, ChatGPT hasn't shown significant signs of slowing down in developing... Jace Dela Cruz, Tech Times 28 February 2023, 08:02 am Every dystopian science fiction starts with the idea that humans will be overrun by robots. 90s movies and books believed this would happen to... ChatGPT is an artificial intelligence (AI) chatbot built on top of OpenAI's foundational large language models (LLMs) like GPT-4 and its predecessors. Aleksandra Yosifova 14 Aug 2023 7 min read ChatGPT isn't the first language model; it isn't even the first GPT model. But it made a significant leap in natural language processing—popularizing large language models and accelerating the adoption of AI. What factors contributed to ChatGPT's success?"

In [46]:
# response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
# response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [47]:
from langchain import hub

response_prompt = hub.pull("langchain-ai/stepback-answer")

In [48]:
chain = {
    # Retrieve context using the normal question
    "normal_context": RunnableLambda(lambda x: x['question']) | retriever,
    # Retrieve context using the step-back question
    "step_back_context": question_gen | retriever,
    # Pass on the question
    "question": lambda x: x["question"]
} | response_prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [49]:
chain.invoke({"question": question})

"No, ChatGPT was not around while Donald Trump was president. ChatGPT was launched by OpenAI on November 30, 2022, which was after Donald Trump's presidency. Therefore, it would not have been available for use during his time in office."

## Baseline

In [50]:
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

{normal_context}

Original Question: {question}
Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [51]:
chain = {
    # Retrieve context using the normal question (only the first 3 results)
    "normal_context": RunnableLambda(lambda x: x['question']) | retriever,
    # Pass on the question
    "question": lambda x: x["question"]
} | response_prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [52]:
chain.invoke({"question": question})

"Based on the given context, it is unclear whether ChatGPT was around while Trump was president. The context mentions that a Twitter user requested ChatGPT to write a positive poem about former President Donald Trump, to which the chatbot declined. However, it does not explicitly state when this incident occurred. Therefore, without further information, we cannot definitively determine whether ChatGPT was available during Trump's presidency."